In [1]:
import pandas as pd
import numpy as np
from collections import Counter
from sklearn.preprocessing import LabelEncoder
import pandas_profiling as ppf

####  ================================== step_1/5 读取数据时将ID作为index ========================================

In [2]:
# base信息
base_train_sum=pd.read_csv(r'./data/base_train_sum.csv',encoding='gb2312').reset_index(drop=True).set_index('ID')
base_varify=pd.read_csv(r'./data/base_verify1.csv',encoding='gb2312').reset_index(drop=True).set_index('ID')
knowledge_train=pd.read_csv(r'./data/knowledge_train_sum.csv',encoding='gb2312').reset_index(drop=True).set_index('ID')
paient_information_verify1=pd.read_csv(r'./data/paient_information_verify1.csv',encoding='gb2312').reset_index(drop=True).set_index('ID')
# money信息
money_report_train_sum=pd.read_csv(r'./data/money_report_train_sum.csv',encoding='gb2312').reset_index(drop=True).set_index('ID')
money_information_verify1=pd.read_csv(r'./data/money_information_verify1.csv',encoding='gb2312').reset_index(drop=True).set_index('ID')
year_report_train_sum=pd.read_csv(r'./data/year_report_train_sum.csv',encoding='gb2312').reset_index(drop=True).set_index('ID')
year_report_verify1=pd.read_csv(r'./data/year_report_verify1.csv',encoding='gb2312').reset_index(drop=True).set_index('ID')


####  ========================================= step_2/5 数据合并 ==============================================

In [5]:
# 基本信息的数据
no_year=pd.concat([pd.concat([base_train_sum,base_varify],sort=True).drop(columns='控制人ID'), pd.concat([knowledge_train,paient_information_verify1],sort=True)],axis=1,sort=True)
# no_year

In [36]:
# 三年的有关money的数据
three_year=pd.concat([pd.concat([money_report_train_sum,money_information_verify1]),pd.concat([year_report_train_sum,year_report_verify1]).drop(columns='year')],axis=1)

####  ================================== step_3/5 对money的three_year处理 ========================================

In [37]:
# 把‘year’补充完整,此步较慢，可以保存文件至three_year
pp=[]
for i in list(set(three_year.index)):
    a=three_year[three_year.index==i]['year'].iloc[0]
    b=three_year[three_year.index==i]['year'].iloc[1]
    c=three_year[three_year.index==i]['year'].iloc[2]
    year=[2015,2016,2017]
    real=[a,b,c]
    if set(year)==set(real):
        continue
    else:
        pp.append(i)
# three_year.to_csv('./three_year.csv')
df=three_year[three_year.index==pp[0]]
for i in range(1,len(pp)):
    df=pd.concat([three_year[three_year.index==pp[i]],df])
    
df['year']=[2015.,2016.,2017.]*int(df.shape[0]/3)
# three_year=three_year.combine_first(df)  index唯一时可以用，这里不能用
three_year.drop(index=[i for i in pp],inplace=True)
three_year=pd.concat([three_year,df],sort=True)
three_year.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 137802 entries, 28 to 2490465
Data columns (total 18 columns):
year           137802 non-null float64
主营业务收入         136444 non-null float64
从业人数           136477 non-null float64
债权融资成本         136418 non-null float64
债权融资额度         136464 non-null float64
内部融资和贸易融资成本    136410 non-null float64
内部融资和贸易融资额度    136484 non-null float64
净利润            136369 non-null float64
利润总额           136471 non-null float64
所有者权益合计        136418 non-null float64
纳税总额           136487 non-null float64
股权融资成本         136414 non-null float64
股权融资额度         136450 non-null float64
营业总收入          136382 non-null float64
负债总额           136482 non-null float64
资产总额           136345 non-null float64
项目融资和政策融资成本    136392 non-null float64
项目融资和政策融资额度    136406 non-null float64
dtypes: float64(18)
memory usage: 20.0 MB


In [43]:
# 三年数据分别处理，留下三年ID的交集
money_2015=three_year[three_year['year']==2015].drop(['year'],axis=1)
money_2016=three_year[three_year['year']==2016].drop(['year'],axis=1)
money_2017=three_year[three_year['year']==2017].drop(['year'],axis=1)
# inner0=list(set(money_2015.index).intersection(set(money_2016.index)).intersection(set(money_2017.index)))  
inner=list(set(no_year.index).intersection(set(money_2015.index)))                                                    # base的交集
len(inner)

45931

In [44]:
# 所有空值暂且用平均数填充（所有者权益总额=资产总额-负债总额）
lt=[]  # 记录每一年的数据的形状
for money in [money_2015,money_2016,money_2017]:
    mean=money.describe()
    values={money.columns[0]:mean[mean.columns[0]][1], money.columns[1]:mean[mean.columns[1]][1], money.columns[2]:mean[mean.columns[2]][1], money.columns[3]:mean[mean.columns[3]][1],
           money.columns[4]:mean[mean.columns[4]][1],money.columns[5]:mean[mean.columns[5]][1],money.columns[6]:mean[mean.columns[6]][1],money.columns[7]:mean[mean.columns[7]][1],
           money.columns[8]:mean[mean.columns[8]][1],money.columns[9]:mean[mean.columns[9]][1],money.columns[10]:mean[mean.columns[10]][1],money.columns[11]:mean[mean.columns[11]][1],
           money.columns[12]:mean[mean.columns[12]][1],money.columns[13]:mean[mean.columns[13]][1],money.columns[14]:mean[mean.columns[14]][1],money.columns[15]:mean[mean.columns[15]][1],
           money.columns[16]:mean[mean.columns[16]][1]}
    money.fillna(value=values,inplace=True)
    money['所有者权益合计']=money['资产总额']-money['负债总额']
#     舍弃交集外的ID
    for index in money.index:
        if index in inner:
            continue
        else:
            money.drop(index,axis=0,inplace=True) # inplace指定原地操作
    lt.append(money.shape)
#     对每一年的数据每一列标准化(x-mu)/std
    for i in range(money.shape[1]):
        money[money.columns[i]]=(money[money.columns[i]] - mean[mean.columns[i]][1])/mean[mean.columns[i]][2]
print(lt)

[(45931, 17), (45931, 17), (45931, 17)]


In [45]:
# 三年的数据暂且取平均
average=(money_2015+money_2016+money_2017)/3
average

,主营业务收入,从业人数,债权融资成本,债权融资额度,内部融资和贸易融资成本,内部融资和贸易融资额度,净利润,利润总额,所有者权益合计,纳税总额,股权融资成本,股权融资额度,营业总收入,负债总额,资产总额,项目融资和政策融资成本,项目融资和政策融资额度
ID,,,,,,,,,,,,,,,,,
28,-0.384416,0.063669,-0.376463,-0.376167,-0.226258,-0.226812,-0.391587,-0.247471,-0.004741,-0.463019,0.406379,0.405600,-0.338439,-0.399303,-0.584048,-0.340293,-0.340049
230,-0.505247,-0.102344,-0.376463,-0.376167,-0.131425,-0.132086,-0.305196,-0.402976,0.106861,-0.463019,-0.232299,-0.232355,-0.547484,-0.371525,-0.454299,-0.284360,-0.284125
429,1.398089,-0.215127,0.422401,0.422863,-0.325154,-0.325293,-1.181800,0.637125,-0.461944,-0.463019,1.593755,1.592084,1.565990,1.380232,1.632408,-0.340293,-0.340049
727,1.047925,0.355541,-0.376463,-0.376167,0.276225,0.274462,-1.224502,0.836748,-1.434351,-0.463019,2.129373,2.129724,1.686916,2.114191,1.910589,-0.340293,-0.340049
1137,-0.505274,-0.323164,-0.024595,-0.024152,-0.213408,-0.214017,-0.345021,-0.277656,0.393514,-0.463019,-0.285456,-0.285470,-0.451257,-0.362914,-0.215609,-0.236656,-0.236361
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2884215,0.496968,-0.086906,0.670164,0.670899,-0.325154,-0.325293,-0.785107,0.233498,0.415272,-0.463019,-0.285456,-0.285470,0.608267,0.040525,0.390140,0.676944,0.677640
3015090,0.402190,0.416576,0.161384,0.161671,1.321758,1.317170,-0.902459,0.552677,-0.623410,-0.463019,0.029675,0.030519,0.891865,0.680957,0.487089,-0.340293,-0.340049
5991919,-0.696946,0.087269,-0.376463,-0.376167,-0.241054,-0.241419,-0.149531,-0.589776,0.216414,-0.369505,-0.258391,-0.258332,-0.708450,-0.719911,-0.870371,-0.305747,-0.305487


####  ========================================= step_4/5 对base的no_year处理 =======================================

In [46]:
for index in no_year.index:
        if index in inner:
            continue
        else:
            no_year.drop(index,axis=0,inplace=True) # inplace指定原地操作

In [47]:
no_year.info()

<class 'pandas.core.frame.DataFrame'>
Float64Index: 45931 entries, 28.0 to 5999999.0
Data columns (total 11 columns):
flag       35648 non-null float64
企业类型       45469 non-null object
区域         45506 non-null object
控制人持股比例    45477 non-null float64
控制人类型      45425 non-null object
注册时间       45444 non-null float64
注册资本       45467 non-null float64
行业         45511 non-null object
专利         45472 non-null float64
商标         45457 non-null float64
著作权        45496 non-null float64
dtypes: float64(7), object(4)
memory usage: 4.2+ MB


In [48]:
# 以众数填充
lt2=[]
No_year=no_year.drop(columns='flag')
for i in No_year.columns:
    lt2.append(list(Counter(no_year[i]).most_common(1)[0])[0])

values={'企业类型':lt2[0],'区域':lt2[1],'控制人持股比例':lt2[2],'控制人类型':lt2[3],'注册时间':lt2[4],
        '注册资本':lt2[5],'行业':lt2[6],'专利':lt2[7],'商标':lt2[8],'著作权':lt2[9]}
No_year.fillna(value=values,inplace=True)

In [49]:
no_year=no_year.combine_first(No_year)

In [50]:
# 注册时间以2000年为元年，注册资本标准化(x-mu)/std
no_year['注册时间']=no_year['注册时间']-2000
no_year['注册资本']=(no_year['注册资本']-no_year['注册资本'].mean())/no_year['注册资本'].std()

In [51]:
# 对文本数据编码
usa=no_year[['行业','企业类型','控制人类型','区域']]
for col in ['行业','企业类型','控制人类型','区域']:
    no_year[col] = LabelEncoder().fit_transform(no_year[col])
    print(Counter(usa[col]))
    print(Counter(no_year[col]))

Counter({'交通运输业': 8114, '商业服务业': 7659, '服务业': 7597, '社区服务': 7529, '工业': 7522, '零售业': 7510})
Counter({0: 8114, 1: 7659, 3: 7597, 4: 7529, 2: 7522, 5: 7510})
Counter({'农民专业合作社': 9574, '有限责任公司': 9106, '合伙企业': 9105, '集体所有制企业': 9099, '股份有限公司': 9047})
Counter({0: 9574, 2: 9106, 1: 9105, 4: 9099, 3: 9047})
Counter({'企业法人': 23279, '自然人': 22652})
Counter({0: 23279, 1: 22652})
Counter({'江西': 7039, '广西': 6605, '福建': 6508, '山东': 6460, '广东': 6453, '湖北': 6450, '湖南': 6416})
Counter({3: 7039, 2: 6605, 6: 6508, 0: 6460, 1: 6453, 4: 6450, 5: 6416})


###### 行业
商业服务业：1 ；服务业：3 ；社区服务：4 ；零售业：5 ；工业：2 ；交通运输业:0
###### 区域
福建：   6  ；广西：2  ； 江西：  3  ； 山东：    0  ；湖北： 4  ；湖南：5 ；广东：1 
###### 企业类型
农民专业合作社：0；股份有限公司：3；有限责任公司：2；集体所有制企业：4；合伙企业：1
###### 控制人类型
自然人:1     ;企业法人:0

In [52]:
no_year

,flag,专利,企业类型,区域,商标,控制人持股比例,控制人类型,注册时间,注册资本,著作权,行业
ID,,,,,,,,,,,
28.0,1.0,0.0,0,6,1.0,0.56,0,7.0,-1.036705,1.0,0
230.0,1.0,0.0,0,1,0.0,1.00,0,8.0,-0.577344,0.0,3
429.0,1.0,1.0,4,3,0.0,0.75,1,5.0,1.635303,0.0,2
693.0,1.0,0.0,3,0,0.0,0.98,0,11.0,1.175942,0.0,4
727.0,1.0,0.0,3,3,0.0,0.54,0,1.0,1.302179,0.0,5
...,...,...,...,...,...,...,...,...,...,...,...
5999995.0,0.0,0.0,1,5,1.0,0.83,0,0.0,1.628290,1.0,2
5999996.0,0.0,1.0,0,4,1.0,0.93,1,11.0,-0.994626,0.0,5
5999997.0,0.0,1.0,4,6,1.0,0.73,0,13.0,1.410882,0.0,0


#### =================================== step_5/5 合并base与money ==============================================

In [54]:
encoded_all_data=pd.concat([no_year,average],axis=1,sort=True)
encoded_all_data.info()

<class 'pandas.core.frame.DataFrame'>
Float64Index: 45931 entries, 28.0 to 5999999.0
Data columns (total 28 columns):
flag           35648 non-null float64
专利             45931 non-null float64
企业类型           45931 non-null int32
区域             45931 non-null int32
商标             45931 non-null float64
控制人持股比例        45931 non-null float64
控制人类型          45931 non-null int32
注册时间           45931 non-null float64
注册资本           45931 non-null float64
著作权            45931 non-null float64
行业             45931 non-null int32
主营业务收入         45931 non-null float64
从业人数           45931 non-null float64
债权融资成本         45931 non-null float64
债权融资额度         45931 non-null float64
内部融资和贸易融资成本    45931 non-null float64
内部融资和贸易融资额度    45931 non-null float64
净利润            45931 non-null float64
利润总额           45931 non-null float64
所有者权益合计        45931 non-null float64
纳税总额           45931 non-null float64
股权融资成本         45931 non-null float64
股权融资额度         45931 non-null float64
营业总收入          4

In [55]:
encoded_all_data.to_csv(r'./data/created_data/encoded_all_data.csv')

In [21]:
ppf.ProfileReport(encoded_all_data)